<a href="https://colab.research.google.com/github/rynwhd/Language-Models/blob/main/cipher_decryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Generate a random sub cipher
# Read in Moby Dick, create a char-level lang model
# Encoding and Decoding Functions
# f(message) = log likelihood
#Genetic Algorithm 

In [4]:
import string
import regex as re
import random
import numpy as np
from numpy import log 
import textwrap

In [5]:
!wget -nc https://lazyprogrammer.me/course_files/moby_dick.txt

--2023-02-11 04:06:36--  https://lazyprogrammer.me/course_files/moby_dick.txt
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘moby_dick.txt’

moby_dick.txt           [    <=>             ]   1.17M  1.80MB/s    in 0.7s    

2023-02-11 04:06:37 (1.80 MB/s) - ‘moby_dick.txt’ saved [1227649]



In [39]:
cipher={}
decipher={}
letter1=list(string.ascii_lowercase)
letter2=list(string.ascii_lowercase)
random.shuffle(letter2)
for i in range(26):
  cipher[letter1[i]]=letter2[i]
for i,v in cipher.items():
  decipher[v]=i

In [40]:
def encode(text):
  encoded=''
  for i in text:
    if i!=' ':
      encoded+=(cipher[i.lower()])
    else:
      encoded+=i
  return encoded


In [41]:
def decode(text,map):
  decoded=''
  for i in text:
    if i!=' ':
      decoded+=map[i]
    else:
      decoded+=i
  return decoded

In [42]:
unigram={}
bigram={}

In [43]:
text=[]
reg=re.compile('[^a-zA-Z]')
for line in open('moby_dick.txt'):
  line=line.split('\n')
  line=line[0].lower()
  line=line.rstrip()
  line=reg.sub(' ',line)
  if line:
    text.append(line)

In [44]:
for line in text:
  tokens=line.split(' ')
  for word in tokens:
    for char in word:
      unigram[char.lower()]=unigram.get(char.lower(),0.)+1
    for i in range(len(word)-1):
      bi=word[i].lower()+word[i+1].lower()
      bigram[bi]=bigram.get(bi,0.)+1


In [45]:
bigram_nf={}

In [ ]:
#normalisation to probabilities

In [46]:
pi_len=sum(unigram.values())
for i,v in unigram.items():
  unigram[i]=v/pi_len

In [47]:
bigram_len=sum(bigram.values())
for i,v in bigram.items():
  bigram[i]=v/bigram_len

In [17]:
# def fit(x):
#   if len(x)==1:
#     return unigram[x]
#   else:
#     return bigram[x]

In [104]:
def word_prob(word,add_1_prob):
  if word:
    i=word[0]
    logp=log(unigram[i])
    for j in word[1:]:
      if i+j in bigram:
        logp+=log(bigram[i+j])
      else:
        bigram_nf[i+j]=bigram_nf.get(i+j,0.)+1
        logp+=-add_1_prob
      i=j
    return logp
  else:
    return -add_1_prob

In [105]:
def sentence_prob(sent,add_1_prob):
  logp=0
  for i in sent.split(' '):
    logp+=word_prob(i,add_1_prob)
  return logp

In [62]:
# Using Genetic Algorithms

-80.84004747025958

In [ ]:
#initialization of dna_pool

In [50]:
dna_pool=[]
for _ in range(20):
  dna=list(string.ascii_lowercase)
  random.shuffle(dna)
  dna_pool.append(dna)

In [51]:
def evolve_offspring(dna_pool,n_children):
  offspring=[] #list to collect offsprings
  for dna in dna_pool:
    for i in range(n_children):
      copy=dna.copy()
      j=np.random.randint(len(copy))
      k=np.random.randint(len(copy))
      copy[j],copy[k]=copy[k],copy[j]
      offspring.append(copy)
  return offspring+dna_pool

In [52]:
txt="I then lounged down the street and found, as I expected, that there was a mews in a lane which runs downby one wall of the garden. I lent the ostlers a hand in rubbingdown their horses, and received in exchange twopence, a glass ofhalf-and-half, two fills of shag tobacco, and as much informationas I could desire about Miss Adler, to say nothing of half a dozenother people in the neighbourhood in whom I was not in the leastinterested, but whose biographies I was compelled to listen to."
txt=reg.sub(' ',txt).lower()
encoded_message=encode(txt)


In [114]:
num_iters=1000
scores=np.zeros(num_iters)
best_dna=None
best_map=None
best_score=float('-inf')
for i in range(num_iters):
  if i>0:
    dna_pool=evolve_offspring(dna_pool,10)
  dna2score={}
  for dna in dna_pool:
    current_map={}
    for k,v in zip(letter1,dna): #zip() creates a tuple mapping i.e for we map letter to every alphabet in dna string
      current_map[k]=v
    decoded_message=decode(encoded_message,current_map)
    score=sentence_prob(decoded_message)
    dna2score[''.join(dna)]=score
    if score > best_score:
      best_dna=dna
      best_map=current_map
      best_score=score
  scores[i]=np.mean(list(dna2score.values()))
  sorted_dna=sorted(dna2score.items(),key=lambda x:x[1],reverse=True)
  dna_pool=[list(k) for k,v in sorted_dna[:5]]
  if i%200==0:
    print('iter: ',i,' score:',scores[i],' best so far:',best_score)

In [119]:
d=decode(encoded_message,best_map)

In [120]:
correct=0
incorrect=0
pred=0
mapping={}
for i in range(len(txt)):
  if d[i]==' ':
    pred+=1
    continue
  if d[i]==txt[i]:
    correct+=1
  else:
    mapping[d[i]]=txt[i]
    incorrect+=1

In [85]:
acc=correct/(len(txt)-pred)

In [100]:
acc

In [99]:
encoded_message

In [101]:
d

'i then lounged down the street and found  as i expected  that there was a mews in a lane which runs downby one wall of the garden  i lent the ostlers a hand in rubbingdown their horses  and received in exchange twopence  a glass ofhalf and half  two fills of shag tobacco  and as much informationas i could desire about miss adler  to say nothing of half a dokenother people in the neighbourhood in whom i was not in the leastinterested  but whose biographies i was compelled to listen to '

In [102]:
txt

'i then lounged down the street and found  as i expected  that there was a mews in a lane which runs downby one wall of the garden  i lent the ostlers a hand in rubbingdown their horses  and received in exchange twopence  a glass ofhalf and half  two fills of shag tobacco  and as much informationas i could desire about miss adler  to say nothing of half a dozenother people in the neighbourhood in whom i was not in the leastinterested  but whose biographies i was compelled to listen to '